In [93]:
### Exercice fait main pour les GOTY ###

## 1) Packages et instanciation du client Spark Session
from pyspark.sql import SparkSession
import configparser
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.mllib.linalg import Vectors


import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number
from pyspark.sql.functions import max, min
from pyspark.sql.functions import mean
from pyspark.sql.functions import corr
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.types import IntegerType, DateType, DoubleType

import folium
from folium.plugins import MarkerCluster
import pandas as pd

spark = SparkSession.builder.appName('Brisbane-city-bike').getOrCreate()
sc = spark.sparkContext

In [94]:
## 2) Créer un fichier properties.conf contenant les informations relatives à vos paramètres du programme en dur

# properties.conf = 
# [Bristol-City-bike] 
# Input-data = spark.sql.sources.default(r"C:\Users\Hosti\Desktop\Spark_cours\Code\Bristol-city-bike.json")
# Output-data = spark.sql.sources.default(r"C:\Users\Hosti\Desktop\Spark_cours\Code")
# Kmeans-level= 3  


# config = configparser.ConfigParser()
# path-to-input-data = config['Bristol-City-bike']['Input-data']
# path-to-output-data = config['Bristol-City-bike']['Output-data']
# num-partition-kmeans = config['Bristol-City-bike'][Kmeans-level] 





In [95]:
## 3) Importer le json avec spark
Brisbane = spark.read.json(r"C:\Users\Hosti\Desktop\Spark_cours\Code\Bristol-city-bike.json")
Brisbane.show(3)

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
|Sydney St Ferry T...|-27.474531|153.042728|88 - SYDNEY ST FE...|    88|
+--------------------+----------+----------+--------------------+------+
only showing top 3 rows



In [96]:
## 4) Créer un nouveau data frame Kmeans-df contenant seulement les variables latitude et longitude. 

Kmeans_df = Brisbane["latitude", "longitude"]

Kmeans_df.show(10)

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
|-27.474531|153.042728|
|-27.461881|153.046986|
|-27.469658|153.016696|
| -27.48172| 153.00436|
|-27.493626|153.001482|
|-27.476076|153.002459|
|-27.493963|153.011938|
|-27.482197|153.020894|
+----------+----------+
only showing top 10 rows



In [97]:
## 5) K-means

features = ("latitude", "longitude")
vecteurs = VectorAssembler(inputCols = features, outputCol = "features")
Brisbane_df = vecteurs.transform(Kmeans_df)
Brisbane_df.show()


kmeans = KMeans().setK(3).setSeed(1)  # 3 clusters
model = kmeans.fit(Brisbane_df.select('features'))
fitted = model.transform(Brisbane_df)

+----------+----------+--------------------+
|  latitude| longitude|            features|
+----------+----------+--------------------+
|-27.482279|153.028723|[-27.482279,153.0...|
| -27.47059|153.036046|[-27.47059,153.03...|
|-27.474531|153.042728|[-27.474531,153.0...|
|-27.461881|153.046986|[-27.461881,153.0...|
|-27.469658|153.016696|[-27.469658,153.0...|
| -27.48172| 153.00436|[-27.48172,153.00...|
|-27.493626|153.001482|[-27.493626,153.0...|
|-27.476076|153.002459|[-27.476076,153.0...|
|-27.493963|153.011938|[-27.493963,153.0...|
|-27.482197|153.020894|[-27.482197,153.0...|
|-27.465226|153.050864|[-27.465226,153.0...|
|-27.468447|153.024662|[-27.468447,153.0...|
|-27.473021|153.025988|[-27.473021,153.0...|
|-27.457825|153.036866|[-27.457825,153.0...|
| -27.48148| 153.02368|[-27.48148,153.02...|
|-27.467464|153.022094|[-27.467464,153.0...|
|-27.499963|153.017633|[-27.499963,153.0...|
|-27.490776|152.994747|[-27.490776,152.9...|
|-27.458199|153.041688|[-27.458199,153.0...|
|-27.48180

In [98]:
## 6) Quels sont les noms des colonnes de fitted ? vérifier qu’il s’agit de longitude, latitude, features, predictions.

fitted.show(30)

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[-27.482279,153.0...|         2|
| -27.47059|153.036046|[-27.47059,153.03...|         2|
|-27.474531|153.042728|[-27.474531,153.0...|         1|
|-27.461881|153.046986|[-27.461881,153.0...|         1|
|-27.469658|153.016696|[-27.469658,153.0...|         2|
| -27.48172| 153.00436|[-27.48172,153.00...|         0|
|-27.493626|153.001482|[-27.493626,153.0...|         0|
|-27.476076|153.002459|[-27.476076,153.0...|         0|
|-27.493963|153.011938|[-27.493963,153.0...|         0|
|-27.482197|153.020894|[-27.482197,153.0...|         2|
|-27.465226|153.050864|[-27.465226,153.0...|         1|
|-27.468447|153.024662|[-27.468447,153.0...|         2|
|-27.473021|153.025988|[-27.473021,153.0...|         2|
|-27.457825|153.036866|[-27.457825,153.0...|         1|
| -27.48148| 153.02368|[-27.48148,153.02...|    

In [99]:
## 7) Déterminer les longitudes et latitudes moyennes pour chaque groupe en utilisant spark DSL et SQL. comparer les résultats

fitted.createOrReplaceTempView('Table')

spark.sql("""SELECT MEAN(latitude) as Latitude, MEAN(longitude) as Longitude, prediction FROM Table GROUP BY prediction""").show()


+-------------------+------------------+----------+
|           Latitude|         Longitude|prediction|
+-------------------+------------------+----------+
|-27.460240636363633|153.04186302272726|         1|
| -27.47255990624999|   153.02594553125|         2|
|-27.481218536585374|153.00572882926832|         0|
+-------------------+------------------+----------+



In [118]:
## 8) Bonus:  Faire une visualisation dans une map avec le package leaflet 

m = folium.Map(location=[-27.47130457718122, 153.02508301342277], zoom_start=14)

popup = "Brisbane"
marker_1 = folium.Marker([-27.460240636363633, 153.04186302272726], popup=popup)
marker_2 = folium.Marker([-27.47255990624999, 153.02594553125], popup=popup)
marker_3 = folium.Marker([-27.481218536585374, 153.00572882926832], popup=popup)

m.add_child(marker_1)
m.add_child(marker_2)
m.add_child(marker_3)

m

In [9]:
## 9) Exporter la data frame fitted après élimination de la colonne  features, dans le répertoire path-to-output-data